# Generative Pre-trained Quantum Eigensolver (GPT-QE):

The Generative Quantum Eigensolver (GQE) is a novel method that uses classical generative models to create quantum circuits for quantum simulation. The GQE algorithm optimizes a classical generative model to produce quantum circuits with desired properties, such as a low-energy state.

The [paper](https://arxiv.org/pdf/2401.09253) introduces a specific implementation called the 

generative pre-trained transformer-based quantum eigensolver (GPT-QE), which uses a transformer-based generative model. This model can be trained on existing datasets or without any prior knowledge to find the ground states of electronic structure Hamiltonians.

The GQE approach differs from the Variational Quantum Eigensolver (VQE), because the optimizable parameters are all within the classical generative model, not the quantum circuit itself. This is a key distinction that contributes to the potential scalability and trainability of the GQE model.

## The GQE and VQE Comparison

The main difference between GQE and VQE lies in where the optimizable parameters are located.

- VQE: Parameters are embedded directly within the quantum circuit and are optimized to minimize a cost function.

- GQE: All optimizable parameters are in the classical generative model, which then generates quantum circuits.

This difference in the location of parameters means that the cost function landscapes for the two algorithms are different. By moving the optimization to a classical computer, GQE may be able to overcome trainability issues that VQE faces with larger problems.

<div>
<img src="./images/GQE.png" width="600">
</div>

- Use the logic of the GPT architecture 
- Quantum Gates are analogous to words (tokens)
- Quantum Circuits to sentences 


### How GPT-QE Works
GPT-QE is a specific implementation of GQE that uses a transformer architecture. It draws an analogy between natural language processing and quantum computing.

- Vocabulary and Words: In GPT-QE, the "vocabulary" is a set of unitary operations, and the "words" are the specific unitary operations themselves.

- Grammar and Document: The "grammar" is the set of rules the transformer learns to generate a sequence of operations, and the final sequence of operations forms the "document" or quantum circuit.


- Training: The model is trained so that the circuits it generates have a lower cost value when run on a quantum device. The model learns which sequences of operations are likely to produce a desirable outcome.

The GPT-QE model generates a sequence of tokens, which correspond to unitary operations, to build a quantum circuit. This generation process is based on a parameterized probability distribution, which is trained to favor circuits that lead to low-energy states.

<div>
<img src="./images/gpt-circuit.png" width="600">
</div>

### The Role of Logit-Matching
The training and pre-training of GPT-QE are based on a technique called 

logit-matching. The goal of logit-matching is to make the sum of logits from the generative model approximately equal to the energy of the quantum state produced by the circuit. The cost function is designed to measure how close this approximation is.


- Example 

<div>
<img src="./images/gqe-example.png" width="600">
</div>

## Requirments

In [ ]:
%pip install cudaq-solvers
%pip install cudaq-solvers[gqe]

### Note: 
CUDA-Q Solvers will require the presence of `libgfortran`, which is not distributed with the Python wheel, for provided classical optimizers. If libgfortran is not installed, you will need to install it via your distribution’s package manager. On Debian based systems, you can install this with `apt-get install gfortran`.

In [17]:
import cudaq_solvers as solvers
import cudaq
from cudaq import spin

In [18]:
from lightning.fabric.loggers import CSVLogger
from cudaq_solvers.gqe_algorithm.gqe import get_default_config

### Set the cudaq target: 

In [19]:
try:
    # Set the cudaq target to NVIDIA GPU with fp64 support
    cudaq.set_target('nvidia', option='fp64')
except RuntimeError:
    # If the GPU target is not available, fall back to CPU
    cudaq.set_target('qpp-cpu')


In [20]:
import os, torch

os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
torch.manual_seed(3047)
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Generating the electronic Hamiltonian: 
The CUDA-Q Solvers library accelerates a wide range of applications in the domain of quantum chemistry. To facilitate these calculations, CUDA-Q Solvers provides the `solver.create_molecule` function to allow users to generate basis sets and Hamiltonians for many systems of interest. The molecule class contains basis set informations, and the Hamiltonian `molecule.hamiltonian` for the target systems. These Hamiltonians can then be used as input into the hybrid quantum-classical solvers that the CUDA-Q Solvers API provides.

To learn more, see this [example](https://nvidia.github.io/cudaqx/examples_rst/solvers/molecular_hamiltonians.html)

In [21]:
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., .7474))]
molecule = solvers.create_molecule(geometry, 'sto-3g', 0, 0, casci=True)

spin_ham = molecule.hamiltonian
n_qubits = molecule.n_orbitals * 2
n_electrons = molecule.n_electrons

print(f"Number of qubits: {n_qubits}")
print(f"Number of electrons: {n_electrons}")
print(f"Spin Hamiltonian:\n{spin_ham}")


Number of qubits: 4
Number of electrons: 2
Spin Hamiltonian:
(-0.106477+0i) + (0.17028+0i) * Z0 + (0.17028+0i) * Z1 + (-0.220041+0i) * Z2 + (-0.220041+0i) * Z3 + (0.1202+0i) * Z0Z2 + (0.168336+0i) * Z0Z1 + (0.0454063+0i) * Y0X1X2Y3 + (-0.0454063+0i) * Y0Y1X2X3 + (-0.0454063+0i) * X0X1Y2Y3 + (0.0454063+0i) * X0Y1Y2X3 + (0.165607+0i) * Z0Z3 + (0.1202+0i) * Z1Z3 + (0.165607+0i) * Z1Z2 + (0.174073+0i) * Z2Z3


## Generating the unitary operators: 

These will be the vocabulary for our GPT-QE training. Here, we will use an inspired UCCSD operator pool.

In [23]:
params = [
    0.003125, -0.003125, 0.00625, -0.00625, 0.0125, -0.0125, 0.025, -0.025,
    0.05, -0.05, 0.1, -0.1
]

def pool(params):
    ops = []
    i = 0

    ops.append(
        cudaq.SpinOperator(
            spin.y(i) * spin.z(i + 1) * spin.x(i + 2) * spin.i(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.x(i) * spin.z(i + 1) * spin.y(i + 2) * spin.i(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.i(i) * spin.y(i + 1) * spin.z(i + 2) * spin.x(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.i(i) * spin.x(i + 1) * spin.z(i + 2) * spin.y(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.x(i) * spin.x(i + 1) * spin.x(i + 2) * spin.y(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.x(i) * spin.x(i + 1) * spin.y(i + 2) * spin.x(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.x(i) * spin.y(i + 1) * spin.y(i + 2) * spin.y(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.y(i) * spin.x(i + 1) * spin.y(i + 2) * spin.y(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.x(i) * spin.y(i + 1) * spin.x(i + 2) * spin.x(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.y(i) * spin.x(i + 1) * spin.x(i + 2) * spin.x(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.y(i) * spin.y(i + 1) * spin.x(i + 2) * spin.y(i + 3)))
    ops.append(
        cudaq.SpinOperator(
            spin.y(i) * spin.y(i + 1) * spin.y(i + 2) * spin.x(i + 3)))

    pool = []
    for c in params:
        for op in ops:
            pool.append(c * op)

    return pool


op_pool = pool(params)

print(f"Number of operators in pool: {len(op_pool)}")

Number of operators in pool: 144


In [24]:
def term_coefficients(op: cudaq.SpinOperator) -> list[complex]:
    return [term.evaluate_coefficient() for term in op]


def term_words(op: cudaq.SpinOperator) -> list[cudaq.pauli_word]:
    return [term.get_pauli_word(n_qubits) for term in op]


for i in range(3):
    print(f"Operator {i}:")
    print(f"  Coefficients: {term_coefficients(op_pool[i])}")
    print(f"  Words: {term_words(op_pool[i])}")
    print()

Operator 0:
  Coefficients: [(0.003125+0j)]
  Words: ['YZXI']

Operator 1:
  Coefficients: [(0.003125+0j)]
  Words: ['XZYI']

Operator 2:
  Coefficients: [(0.003125+0j)]
  Words: ['IYZX']



In [25]:
@cudaq.kernel
def kernel(n_qubits: int, n_electrons: int, coeffs: list[float],
           words: list[cudaq.pauli_word]):
    
    # Set the number of qubits 
    q = cudaq.qvector(n_qubits)

    # Initialize the qubits to the HF state.
    for i in range(n_electrons):
        x(q[i])

    # Apply the Pauli operators from the selected pool from GPT model
    for i in range(len(coeffs)):
        exp_pauli(coeffs[i], q, words[i])

In [26]:
def cost(sampled_ops: list[cudaq.SpinOperator], **kwargs):

    full_coeffs = []
    full_words = []

    for op in sampled_ops:
        full_coeffs += [c.real for c in term_coefficients(op)]
        full_words += term_words(op)

    
    
    return cudaq.observe(kernel, spin_ham, n_qubits, n_electrons,
                            full_coeffs, full_words).expectation()

### Running the GQE algorithm:

In [27]:
cfg = get_default_config()
cfg.use_fabric_logging = False
logger = CSVLogger("gqe_h2_logs/gqe.csv")
cfg.fabric_logger = logger
cfg.save_trajectory = False
cfg.verbose = True

# Run GQE
minE, best_ops = solvers.gqe(cost, op_pool, max_iters=25, ngates=10, config=cfg)

print(f'Ground Energy = {minE}')
print('Ansatz Ops')
for idx in best_ops:
    # Get the first (and only) term since these are simple operators
    term = next(iter(op_pool[idx]))
    print(term.evaluate_coefficient().real, term.get_pauli_word(n_qubits))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
INFO: Seed set to 3047
INFO:lightning.fabric.utilities.seed:Seed set to 3047


total trainable params: 85.95M
epoch 0 loss tensor(0.2186, device='cuda:0', grad_fn=<MseLossBackward0>) model.train_step time: 0.7727482318878174 tensor(-1.1154)
epoch 1 loss tensor(0.0613, device='cuda:0', grad_fn=<MseLossBackward0>) model.train_step time: 0.26561498641967773 tensor(-1.1208)
epoch 2 loss tensor(0.1167, device='cuda:0', grad_fn=<MseLossBackward0>) model.train_step time: 0.26775264739990234 tensor(-1.1049)
epoch 3 loss tensor(0.1094, device='cuda:0', grad_fn=<MseLossBackward0>) model.train_step time: 0.26793456077575684 tensor(-1.1308)
epoch 4 loss tensor(0.1481, device='cuda:0', grad_fn=<MseLossBackward0>) model.train_step time: 0.26700305938720703 tensor(-1.1308)
epoch 5 loss tensor(0.0406, device='cuda:0', grad_fn=<MseLossBackward0>) model.train_step time: 0.298445463180542 tensor(-1.1352)
epoch 6 loss tensor(0.4029, device='cuda:0', grad_fn=<MseLossBackward0>) model.train_step time: 0.28936052322387695 tensor(-1.0833)
epoch 7 loss tensor(0.2086, device='cuda:0', gra